# Σε αυτό το Notebook πραγματοποιείται η εκπαίδευση του μοντέλου


In [12]:
#Data Proccesing
import pandas as pd
#Model
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Input, Dense,add
from tensorflow.keras.models import Model

Project_Path='/Users/nickkarras/PycharmProjects/ParkingViolationPredictionGraph_Git'

Αρχικά φορτώνω το Dataset 'Legal_Illegal", το αρχείο με τον καιρό "Final_Weather_Data", και το αρχείο με τις αποστάσεις των τομέων από τα 19 σημεία ενδιαφέροντος 'Distance_Data'. Χωρίζω το Dataset σε 80% train-set και 20% test-set και έπειτα τα σώζω σε ξεχωριστά αρχεία ώστε να έχω πάντα το ίδιο train-set και test-set για να ξέρω άν η κάθε αλλαγή βοηθάει. 

In [13]:
Distance_Data=pd.read_csv(Project_Path+ '/Data/Distance.csv',sep=',',index_col=0)
Final_Weather_Data=pd.read_csv(Project_Path+ '/Data/Final_Weather_Data.csv',low_memory=False,sep=',',index_col=0)
Legal_illegal=pd.read_csv(Project_Path+ '/Data/Scan_Data_Reg_2.3.csv',sep=',',index_col=0)

Legal_illegal=Legal_illegal.loc[Legal_illegal['Key'] < '2020-01-01']
Legal_illegal=Legal_illegal.dropna()
Legal_illegal.to_csv(Project_Path+ '/Data/Full_TimeSeries.csv')

train_data=Legal_illegal.loc[Legal_illegal['Key'] < '2019-10-01']
test_data=Legal_illegal.loc[Legal_illegal['Key'] > '2019-10-01']

train_data.to_csv(Project_Path+ '/Data/Train_TimeSeries.csv')
test_data.to_csv(Project_Path+ '/Data/Test_TimeSeries.csv')

Τα παρακάτω κελιά αποτελούν συναρτήσεις που χρησιμοποιούνται σε όλο το αρχείο  κατά την διάρκεια της εκπαίδευσης 

Αυτή η συνάρτηση δέχεται train-set και test-set. Εφαρμόζει σε αυτά standardization και τα επιστρέφει 

In [14]:
def Scaller(Train,Test):
    Standar_Scaller = StandardScaler()
    Scalled_Train_data=Standar_Scaller.fit_transform(Train)
    Scalled_val_data = Standar_Scaller.transform(Test)
    return Scalled_Train_data,Scalled_val_data

In [18]:
def scheduler(epoch, lr):
    if epoch < 8:
        return lr
    else:
        return lr * tf.math.exp(-0.25)

callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

Δηλώνω Initializers για τους kernel των κρυφών στρωμάτων και τις εξόδου 

In [15]:
Kernel_Sigmoid_Initializer = tf.keras.initializers.GlorotUniform()
Kernel_Relu_Initializer =tf.keras.initializers.HeUniform()

Αυτή η συνάρτηση κατασκευάζει το Residual Neural Networks. Δέχεται το μέγεθος των χαρακτηριστικών ωστε να το χρησιμοποιήσει σαν μέγεθος εισόδου. Δέχεται μια μεταβλητή που θα την χρησιμοποιήσει για να αρχικοποιήσει το bias του επιπέδου εξόδου 

In [16]:

def Residual_NN(Input_Shape,Bias_Sigmoid_Initializer):
    Input_Layer = Input(shape=(Input_Shape,))
    #Dense_Layer1 = Dense(256, activation='relu',kernel_initializer=Kernel_Relu_Initializer)(Input_Layer)
    Dense_Layer2 = Dense(128, activation='relu',kernel_initializer=Kernel_Relu_Initializer)(Input_Layer)
    Dense_Layer3 = Dense(64, activation='relu',kernel_initializer=Kernel_Relu_Initializer)(Dense_Layer2)
    Dense_Layer4 = Dense(32, activation='relu',kernel_initializer=Kernel_Relu_Initializer)(Dense_Layer3)
    Dense_Layer5 = Dense(64, activation='relu',kernel_initializer=Kernel_Relu_Initializer)(Dense_Layer4)
    Residual_Add = add([Dense_Layer3, Dense_Layer5])
    Dense_Layer6 = Dense(16, activation='relu',kernel_initializer=Kernel_Relu_Initializer)(Residual_Add)
    Output_Layer = Dense(1, activation='sigmoid',kernel_initializer=Kernel_Sigmoid_Initializer,bias_initializer=Bias_Sigmoid_Initializer)(Dense_Layer6)

    
    model2 = Model(inputs=Input_Layer, outputs=Output_Layer)
    sgd=tf.keras.optimizers.SGD(learning_rate=0.01)
    model2.compile(optimizer='adamax', loss='mse', metrics='mae')
    return model2

# Training procces

In [17]:
def Prepare_Dataset(Dataset): 
    Dataset=Dataset.drop(['Time_Int'], axis=1)
    a=Dataset['Slot_Timeint']
    b=Dataset['Ilegality_Rate']
    Dataset=Dataset.drop(['Slot_Timeint'], axis=1)
    Dataset=Dataset.drop(['Ilegality_Rate'], axis=1)
    Dataset.insert(8, "Real_Time", a, True)
    Dataset.insert(9, "Real_Rate", b, True)
    
    Dataset=pd.merge(Dataset, Final_Weather_Data, on='Key')
    Dataset=Dataset.drop(['Key'], axis=1)
    
    Dataset=pd.merge(Dataset, Distance_Data, on='Slot_id')
    Dataset=Dataset.drop(['Slot_id'], axis=1)
    return Dataset

Η παρακάτω συνάρτηση εφαρμόζει τεχνική εκθετικής μείωσης στο learning rate μετά την εποχή 40

In [19]:
train_data=pd.read_csv(Project_Path+ '/Data/Train_TimeSeries.csv',sep=',',index_col=0)
test_data=pd.read_csv(Project_Path+ '/Data/Test_TimeSeries.csv',sep=',',index_col=0)


#Καλεί την συνάρτηση ώστε να κάνει τις τελικές αλλάγες στο dataset ώστε να είναι έτοιμο για εκπαίδευση
TestDF=Prepare_Dataset(test_data)
TrainDF=Prepare_Dataset(train_data)

#Χωρίζει σε στόχο και χαρακτηριστικά
train_targets=TrainDF['Real_Rate']
train_data=TrainDF.drop(['Real_Rate'], axis=1)
test_targets=TestDF['Real_Rate']
test_data=TestDF.drop(['Real_Rate'], axis=1)


#Βρίσκει το μέσο των στόχων και το χρησιμοποεί για να αρχικοποιήσει τα bias οταν δημιουργείται το μοντέλο 
Target=train_targets
Bias_Initial_Out=Target.mean()
Bias_Initializer=tf.keras.initializers.Constant(Bias_Initial_Out)

train_data,test_data=Scaller(train_data,test_data)

Εκπαιδεύω για 10 εποχές

In [20]:
model = Residual_NN(Input_Shape=train_data.shape[1],Bias_Sigmoid_Initializer=Bias_Initializer)
model.fit(train_data,train_targets,epochs=10, batch_size=16, verbose=1)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)
model.save(Project_Path + '/DNN_Regressor')

Epoch 1/10
5555/5555 [==============================] - 2s 373us/step - loss: 0.0594 - mae: 0.1951
Epoch 2/10
5555/5555 [==============================] - 2s 384us/step - loss: 0.0541 - mae: 0.1845
Epoch 3/10
5555/5555 [==============================] - 2s 402us/step - loss: 0.0518 - mae: 0.1795
Epoch 4/10
5555/5555 [==============================] - 2s 381us/step - loss: 0.0501 - mae: 0.1756
Epoch 5/10
5555/5555 [==============================] - 2s 372us/step - loss: 0.0487 - mae: 0.1728
Epoch 6/10
5555/5555 [==============================] - 2s 371us/step - loss: 0.0478 - mae: 0.1705
Epoch 7/10
5555/5555 [==============================] - 2s 371us/step - loss: 0.0468 - mae: 0.1687
Epoch 8/10
5555/5555 [==============================] - 2s 375us/step - loss: 0.0461 - mae: 0.1672
Epoch 9/10
5555/5555 [==============================] - 2s 384us/step - loss: 0.0455 - mae: 0.1657
Epoch 10/10
5555/5555 [==============================] - 2s 385us/step - loss: 0.0449 - mae: 0.1644
